![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [5]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

# SOURCE: main table
bank_marketing_df = pd.read_csv('bank_marketing.csv')
print(bank_marketing_df.dtypes)


# client table
client = bank_marketing_df[['client_id','age','job','marital','education','credit_default','mortgage']]
# Replace string '.' with '_'
client['job'] = client['job'].str.replace('.', '_')
client['education'] = client['education'].str.replace('.', '_')
# Replace string 'unknown' with NaN
client['education'] = client['education'].replace('unknown', np.NaN)
client['credit_default'] = client['credit_default'].astype('bool')
client['mortgage'] = client['mortgage'].astype('bool')
print(client.dtypes)


# campaign table
campaign = bank_marketing_df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','month','day']]
# Change data types for 'previous_outcome' & 'campaign_outcome' fields to bool
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')
# Re-define months from mmm format to mm format, as string data types
month_mmm_str = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
month_mm_str = ['01','02','03','04','05','06','07','08','09','10','11','12']
# Replace values from mmm format to mm format
campaign['month'] = campaign['month'].replace(month_mmm_str, month_mm_str)
# In order to use len to identify single values, will require to change data types from int to string
campaign['day'] = campaign['day'].astype('string')
# Lambda as function that is applied to 'day' field to transform data by adding leading zeros to single day values
campaign['day'] = campaign['day'].apply(lambda x: '0' + x if len(x)<2 else x)
# Concat year, month, date & changing data type from string to datetime
campaign['last_contact_date'] = '2022' + '-' + campaign['month'] + '-' + campaign['day']
campaign['last_contact_date'] = campaign['last_contact_date'].astype('datetime64[ns]')
# Drop month, day fields to fulfil requirements
campaign.drop(['day','month'], axis='columns', inplace=True)
print(campaign.dtypes)


# economics table
economics = bank_marketing_df[['client_id','cons_price_idx','euribor_three_months']]

# csv exports
client.to_csv('client.csv')
campaign.to_csv('campaign.csv')
economics.to_csv('economics.csv')

client_id                       int64
age                             int64
job                            object
marital                        object
education                      object
credit_default                 object
mortgage                       object
month                          object
day                             int64
contact_duration                int64
number_contacts                 int64
previous_campaign_contacts      int64
previous_outcome               object
cons_price_idx                float64
euribor_three_months          float64
campaign_outcome               object
dtype: object
client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object
client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_